
Python-based agentic AI application that automatically collects, parses, and stores smart contract vulnerability data from the SWC Registry
Aim: build a structured dataset from markdown documentation on GitHub.

**Agentic AI** refers to systems where a language model (like GPT or Qwen) acts as an "agent" capable of:

* **Planning**: Understanding a complex instruction.
* **Tool use**: Calling external functions or tools to achieve a goal.
* **Autonomy**: Executing a sequence of actions to fulfill a user’s high-level request.

---

## 🧠 How This Code Uses Agentic AI

### 1. **Tools Defined via `@tool`**

Each function (`fetch_markdown`, `parse_markdown`, `store_entry`) is wrapped with `@tool`, making it callable by the agent.

### 2. **Agent Defined via `CodeAgent`**

```python
manager = CodeAgent(
    tools=[fetch_markdown, parse_markdown, store_entry],
    model=model,
)
```

This agent:

* Receives a **natural language instruction** (e.g., `"Fetch, parse, and store SWC-101"`),
* Uses the language model (Qwen) to decide:

  * Which tools to call,
  * In what order,
  * With what inputs.

### 3. **Natural Language Task Execution**

```python
result = manager.run(f"Fetch, parse, and store {swc_id}")
```

This is where the **agentic behavior happens**: the agent interprets that high-level request, plans the actions, and executes the tools step by step — all driven by the LLM.

---

## ⚙️ Under the Hood (Example Workflow)

For `SWC-101`, the agent might:

1. Call `fetch_markdown("SWC-101")`.
2. Take the result and pass it to `parse_markdown(...)`.
3. Store the result via `store_entry(...)`.

This whole orchestration is **not hard-coded** — it's determined **dynamically** by the model interpreting the task.

---



In [6]:
!pip install markdownify duckduckgo-search smolagents --upgrade -q

In [38]:
from huggingface_hub import notebook_login

notebook_login()

In [47]:
import re
import json
import requests
import os
from smolagents import tool, CodeAgent, InferenceClientModel

# Constants
GITHUB_RAW_BASE = "https://raw.githubusercontent.com/SmartContractSecurity/SWC-registry/master/entries/docs"
OUTPUT_PATH = "swc_contracts.json"

# Tool 1: Fetch Markdown
@tool
def fetch_markdown(swc_id: str) -> str:
    """Fetches markdown content for a given SWC ID.

    Args:
        swc_id: The SWC ID (e.g., "SWC-100") to fetch the markdown for.
    """
    url = f"{GITHUB_RAW_BASE}/{swc_id}.md"
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return f"Error fetching {swc_id}: {response.status_code}"

# Tool 2: Parse Markdown
@tool
def parse_markdown(markdown_content: str, swc_id: str) -> dict:
    """Parses SWC markdown content to extract description and solidity code using filenames.

    Args:
        markdown_content: The markdown content as a string.
        swc_id: The SWC ID associated with the markdown content.
    """
    # Extract the description section
    description = re.findall(r'(?s)(?<=\n## Description\n)(.*?)(?=\n##|$)', markdown_content)
    description = [desc.strip() for desc in description if desc.strip()]

    # Strictly match only code blocks directly after filename headers
    pattern = r'###\s+([^\n]+\.sol)\s*\n+```solidity\n(.*?)```'
    matches = re.findall(pattern, markdown_content, re.DOTALL)

    result = {
        "id": swc_id,
        "description": description,
    }

    for filename, code in matches:
        result[filename.strip()] = code.strip()

    return result

# Tool 3: Store Entry
@tool
def store_entry(entry: dict, path: str = OUTPUT_PATH) -> str:
    """Stores or appends the entry to a JSON file.

    Args:
        entry: The dictionary representing the SWC entry.
        path: The path to the JSON file.
    """
    if os.path.exists(path):
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    else:
        data = []

    data.append(entry)

    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

    return f"Saved {entry['id']} to {path}"

# Model and Agent
model = InferenceClientModel("Qwen/Qwen2.5-7B-Instruct")
manager = CodeAgent(
    tools=[fetch_markdown, parse_markdown, store_entry],
    model=model,
)

# Loop through SWC IDs
for swc_num in range(128, 129):
    swc_id = f"SWC-{swc_num}"
    print(f"Processing {swc_id}...")
    try:
        result = manager.run(f"Fetch, parse, and store {swc_id}")
        print(result)
    except Exception as e:
        print(f"Failed to process {swc_id}: {e}")


Processing SWC-128...


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Fetch, parse, and store SWC-128                                                                                 │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-7B-Instruct ───────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID:
Root=1-683d6d66-0ffd051f503109315fb0c389;e9d310ad-c834-4fa4-9a6b-90d23f30eac0)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 1: Duration 0.04 seconds]

Failed to process SWC-128: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-683d6d66-0ffd051f503109315fb0c389;e9d310ad-c834-4fa4-9a6b-90d23f30eac0)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.
